In [1]:
from simplesim.risk import Risk
from simplesim.chain import Chain
from simplesim.module import Module
from simplesim.product import Product
from simplesim.csv_input import CSVInput

risk_list = [Risk('aaa', probability=0.2, recover_time=3, gradual_recover=True),
             Risk('aaa', probability=0.1, recover_time=10, gradual_recover=True),
             Risk('aaa', probability=0.05, recover_time=100, gradual_recover=True),
             (Risk('aaa', probability=0.1, recover_time=10), Risk('aaa', probability=0.1, recover_time=10),
              Risk('aaa', probability=0.1, recover_time=10)),
             Risk('aaa', probability=0.01, recover_time=100, gradual_recover=True),
             Risk('aaa', probability=0.001, recover_time=1000, gradual_recover=True),
             Risk('aaa', probability=0.1, recover_time=1)
             ]

In [2]:
# Input


data_dir = CSVInput('data')
data_dir.get_chain()



success {'sss': <simplesim.module.Module object at 0x0000026393380850>, 'ssa': <simplesim.module.Module object at 0x00000263933811E0>, 'ssb': <simplesim.module.Module object at 0x0000026393383D90>, 'ssc': <simplesim.module.Module object at 0x0000026393383DC0>, 'dde': <simplesim.module.Module object at 0x0000026393380100>, 'ddr': <simplesim.module.Module object at 0x0000026393380640>, 'ddt': <simplesim.module.Module object at 0x0000026393383CD0>, 'ddy': <simplesim.module.Module object at 0x00000263932C6740>, 'ddu': <simplesim.module.Module object at 0x00000263932C7730>, 'ffr': <simplesim.module.Module object at 0x00000263932C74F0>, 'ffg': <simplesim.module.Module object at 0x00000263932C6EF0>, 'ffb': <simplesim.module.Module object at 0x000002639330C2E0>}
